# Chapter 14: Rebalancing and Tax Loss Harvesting

In [ ]:
# Learn how to calculate Portfolio Drift 

In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
class Allocation:
  def __init__(self, ticker, percentage):
    self.ticker = ticker
    self.percentage = percentage
    self.units = 0.0

class Portfolio:

  def __init__(self, tickerString: str, expectedReturn: float, portfolioName: str, riskBucket: int):

    self.name = portfolioName
    self.riskBucket = riskBucket
    self.expectedReturn = expectedReturn
    self.allocations = []

    from pypfopt.efficient_frontier import EfficientFrontier
    from pypfopt import risk_models
    from pypfopt import expected_returns

    df = self.__getDailyPrices(tickerString, "20y")

    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)

    ef = EfficientFrontier(mu, S)

    ef.efficient_return(expectedReturn)
    self.expectedRisk = ef.portfolio_performance()[1]
    portfolioWeights = ef.clean_weights()

    for key, value in portfolioWeights.items():
      newAllocation = Allocation(key, value)
      self.allocations.append(newAllocation)

  def __getDailyPrices(self, tickerStringList, period):
    data = yf.download(tickerStringList, group_by="Ticker", period=period)
    data = data.iloc[:, data.columns.get_level_values(1)=="Close"]
    data = data.dropna()
    data.columns = data.columns.droplevel(1)
    return data

  def printPortfolio(self):
    print("Portfolio Name: " + self.name)
    print("Risk Bucket: " + str(self.riskBucket))
    print("Expected Return: " + str(self.expectedReturn))
    print("Expected Risk: " + str(self.expectedRisk))
    print("Allocations: ")
    for allocation in self.allocations:
      print("Ticker: " + allocation.ticker + ", Percentage: " + str(allocation.percentage))

  @staticmethod
  def getPortfolioMapping(riskToleranceScore, riskCapacityScore):
    allocationLookupTable=pd.read_csv('./Data/Risk Mapping Lookup.csv')
    matchTol = (allocationLookupTable['Tolerance_min'] <= riskToleranceScore) & (allocationLookupTable['Tolerance_max'] >= riskToleranceScore)
    matchCap = (allocationLookupTable['Capacity_min'] <= riskCapacityScore) & (allocationLookupTable['Capacity_max'] >= riskCapacityScore)
    portfolioID = allocationLookupTable['Portfolio'][(matchTol & matchCap)]
    return portfolioID.values[0]

class Goal:
  def __init__(self, name, targetYear, targetValue, initialContribution=0, monthlyContribution=0, priority=""):
    self.name = name
    self.targetYear = targetYear
    self.targetValue = targetValue
    self.initialContribution = initialContribution
    self.monthlyContribution = monthlyContribution
    if not (priority == "") and not (priority in ["Dreams", "Wishes", "Wants", "Needs"]):
            raise ValueError('Wrong value set for Priority.')
    self.priority = priority

  def getGoalProbabilities(self):
    if (self.priority == ""):
            raise ValueError('No value set for Priority.')
    lookupTable=pd.read_csv('./Data/Goal Probability Table.csv')
    match = (lookupTable['Realize'] == self.priority)
    minProb = lookupTable['MinP'][(match)]
    maxProb = lookupTable['MaxP'][(match)]
    return minProb.values[0], maxProb.values[0]

class AccountType():
  def __init__(self, value: str):
    if not value in("Taxable", "Roth IRA", "Traditional IRA"):
      raise ValueError("Allowed types: Taxable, Roth IRA, Traditional IRA")
    self.value = value
  def __eq__(self, other):
      return self.value == other.value

class AccountStatus():
  def __init__(self, value: str):
    if not value in("PENDING", "IN_REVIEW", "APPROVED", "REJECTED", "SUSPENDED"):
      raise ValueError("Allowed statuses: PENDING, IN_REVIEW, APPROVED, REJECTED, SUSPENDED")
    self.value = value
  def __eq__(self, other):
      return self.value == other.value

class Account():
  def __init__(self, number: str, accountType: AccountType, accountStatus: AccountStatus, cashBalance: float=0.0):
    self.goals = []
    self.number = number
    self.cashBalance = cashBalance
    self.accountType = accountType
    self.accountStatus = accountStatus

class TransactionType():
  def __init__(self, value: str):
    if not value in("BUY", "SELL"):
      raise ValueError("Allowed types: BUY, SELL.")
    self.value = value
  def __eq__(self, other):
      return self.value == other.value

class OrderStatus():
  def __init__(self, value: str):
    if not value in("NEW", "PENDING", "FILLED", "REJECTED"):
      raise ValueError("Allowed statuses: NEW, PENDING, FILLED, REJECTED.")
    self.value = value
  def __eq__(self, other):
      return self.value == other.value

class Order:
  def __init__(self, account: Account, goal: Goal, transactionType: TransactionType, status: OrderStatus=OrderStatus("NEW"), dollarAmount: float=0.0):
    
    self.account = account
    self.transactionType = transactionType
    self.dollarAmount = dollarAmount
    self.goal = goal
    self.status = status

  def checkAccountStatus(self) -> bool:
    if self.account.accountStatus == AccountStatus("APPROVED"):
      return True
    else:
      return False

  def checkOrderSize(self) -> bool:
    if self.dollarAmount > 1.00:
      return True
    else:
      return False

  def checkBalances(self) -> bool:
    if self.transactionType == TransactionType("BUY") and self.account.cashBalance >= self.dollarAmount:
      return True
    elif self.transactionType == TransactionType("SELL"):
      goalValue = 0.0
      for allocation in goal.portfolio.allocations:
        price = float(yf.Ticker(allocation.ticker).info["previousClose"])
        goalValue += allocation.units * price
      if self.dollarAmount <= goalValue:
        return True
      else:
        return False
    else:
      return False

  def checkOrderViability(self) -> bool:
    if self.checkAccountStatus() and self.checkOrderSize() and self.checkBalances() and isMarketOpen():
      return True
    else:
      return False

  def split(self) -> list:
    splits = []
    for allocation in self.goal.portfolio.allocations:
      if (allocation.percentage > 0):
        splits.append(SplitOrder(originalOrder=self, ticker=allocation.ticker, dollarAmount=allocation.percentage * self.dollarAmount))
    return splits

In [ ]:
myPortfolio = Portfolio("VTI TLT IEI GLD DBC", expectedReturn = 0.05, portfolioName = "Moderate", riskBucket = 3)
myGoal = Goal(name="Vacation", targetYear=2027, targetValue=10000, priority="Dreams", portfolio=myPortfolio)
myAccount=Account(number="123456789", accountType="Taxable", accountStatus=AccountStatus("APPROVED"), cashBalance=11.0)
myAccount.goals.append(myGoal)

In [ ]:
myPortfolio2 = Portfolio("VTI TLT IEI GLD DBC", expectedReturn = 0.03, portfolioName = "Conservative", riskBucket = 2)
myGoal2 = Goal(name="Car", targetYear=2025, targetValue=5000, priority="Dreams", portfolio=myPortfolio2)
myAccount2=Account(number="987654321", accountType="Taxable", accountStatus=AccountStatus("APPROVED"), cashBalance=21.0)
myAccount2.goals.append(myGoal2)

In [ ]:
allocations = [obj.percentage for obj in myPortfolio2.allocations]
allocations

[0.63759, 0.0, 0.0, 0.16638, 0.19603]

In [ ]:
holdings = [obj.units for obj in myPortfolio2.allocations]
holdings

[0.11394768463229901, 0.0, 0.0, 0.019544835304941135, 0.019037328311973366]

In [ ]:
# Get Portfolio data
market_values = []
for allocation in myPortfolio.allocations:
  price = float(yf.Ticker(allocation.ticker).info["previousClose"])
  market_values.append(price)
market_values

[116.53, 107.09, 25.09, 167.26, 203.99]

In [ ]:
# Define the model portfolio allocations
#allocations = [0.3, 0.2, 0.4, 0.1]
allocations = [obj.percentage for obj in myPortfolio.allocations]

# Define the current holdings and market values of the assets in the portfolio
#holdings = [100, 200, 150, 50]
holdings = [obj.units for obj in myPortfolio.allocations]

#market_values = [10, 20, 30, 40]
market_values = []
for allocation in myPortfolio.allocations:
  price = float(yf.Ticker(allocation.ticker).info["previousClose"])
  market_values.append(price)

In [ ]:
# Calculate the current allocation of assets in the portfolio
import numpy as np
current_allocation = []
for i in range(len(holdings)):
    current_allocation.append(holdings[i] * market_values[i])
current_allocation = [x / sum(current_allocation) for x in current_allocation]
print(current_allocation)

# Determine the difference between the model portfolio allocations and the current allocation of assets
diff = [x1 - x2 for (x1, x2) in zip(allocations, current_allocation)]

print("Portfolio Drift: " + '{0:.2f}'.format((np.abs(diff).sum()/2)*100) + "%")

[0.19620313782668816, 0.10408459755628671, 0.0, 0.3614762574927466, 0.3382360071242787]
Portfolio Drift: 1.15%


In [ ]:
class Portfolio:

  def __init__(self, tickerString: str, expectedReturn: float, portfolioName: str, riskBucket: int):

    self.name = portfolioName
    self.riskBucket = riskBucket
    self.expectedReturn = expectedReturn
    self.allocations = []
    self.needRebalancing = False

    from pypfopt.efficient_frontier import EfficientFrontier
    from pypfopt import risk_models
    from pypfopt import expected_returns

    df = self.__getDailyPrices(tickerString, "20y")

    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)

    ef = EfficientFrontier(mu, S)

    ef.efficient_return(expectedReturn)
    self.expectedRisk = ef.portfolio_performance()[1]
    portfolioWeights = ef.clean_weights()

    for key, value in portfolioWeights.items():
      newAllocation = Allocation(key, value)
      self.allocations.append(newAllocation)

  def __getDailyPrices(self, tickerStringList, period):
    data = yf.download(tickerStringList, group_by="Ticker", period=period)
    data = data.iloc[:, data.columns.get_level_values(1)=="Close"]
    data = data.dropna()
    data.columns = data.columns.droplevel(1)
    return data

  def printPortfolio(self):
    print("Portfolio Name: " + self.name)
    print("Risk Bucket: " + str(self.riskBucket))
    print("Expected Return: " + str(self.expectedReturn))
    print("Expected Risk: " + str(self.expectedRisk))
    print("Allocations: ")
    for allocation in self.allocations:
      print("Ticker: " + allocation.ticker + ", Percentage: " + str(allocation.percentage))

  @staticmethod
  def getPortfolioMapping(riskToleranceScore, riskCapacityScore):
    allocationLookupTable=pd.read_csv('./Data/Risk Mapping Lookup.csv')
    matchTol = (allocationLookupTable['Tolerance_min'] <= riskToleranceScore) & (allocationLookupTable['Tolerance_max'] >= riskToleranceScore)
    matchCap = (allocationLookupTable['Capacity_min'] <= riskCapacityScore) & (allocationLookupTable['Capacity_max'] >= riskCapacityScore)
    portfolioID = allocationLookupTable['Portfolio'][(matchTol & matchCap)]
    return portfolioID.values[0]

  def calculateDiffsToModel(self) -> list:
    allocations = [obj.percentage for obj in self.allocations]
    holdings = [obj.units for obj in self.allocations]
    if sum(holdings) == 0.0:
      return []
    market_values = []
    for allocation in self.allocations:
      price = float(yf.Ticker(allocation.ticker).info["previousClose"])
      market_values.append(price)
    
    current_allocation = []
    for i in range(len(holdings)):
        current_allocation.append(holdings[i] * market_values[i])
    current_allocation = [x / sum(current_allocation) for x in current_allocation]
    
    diff = [x1 - x2 for (x1, x2) in zip(allocations, current_allocation)]
    return diff

  def checkNeedRebalancing(self, thres: float, diff: list=[]):
    if diff == []:
      diff = self.calculateDiffsToModel()
    drift = self.calculateDrift(diff)
    
    if drift >= thres:
      self.needRebalancing = True
    else:
      self.needRebalancing = False

  def calculateDrift(self, diff: list=[]) -> float:
    if diff == []:
      diff = self.calculateDiffsToModel()
    return(np.abs(diff).sum()/2)

  def rebalance(self, diff: list=[]) -> list:
    if diff == []:
      diff = self.calculateDiffsToModel()

    if not self.needRebalancing:
      return []

    splitOrders = []
    for i in range(len(diff)):
        if diff[i] > 0:
            diffValue = diff[i] * holdings[i] * market_values[i]
            newOrder = Order(account = myAccount, 
                                goal = myGoal, 
                                transactionType = TransactionType('BUY'), 
                                dollarAmount = diffValue)
            splitOrders.append(SplitOrder(originalOrder = newOrder,
                                          ticker = myPortfolio.allocations[i].ticker, 
                                          dollarAmount = diffValue))
        elif diff[i] < 0:
            diffValue = abs(diff[i]) * holdings[i] * market_values[i]
            newOrder = Order(account = myAccount, 
                                goal = myGoal, 
                                transactionType = TransactionType('SELL'), 
                                dollarAmount = diffValue)
            splitOrders.append(SplitOrder(originalOrder = newOrder,
                                          ticker = myPortfolio.allocations[i].ticker, 
                                          dollarAmount = diffValue))
    return splitOrders

class SplitOrder:
  def __init__(self, originalOrder: Order, ticker: str, dollarAmount: float):
    
    self.originalOrder = originalOrder
    self.ticker = ticker
    self.dollarAmount = dollarAmount
    self.units = 0
    

In [ ]:
#myPortfolio = Portfolio("VTI TLT IEI GLD DBC", expectedReturn = 0.05, portfolioName = "Moderate", riskBucket = 3)
#myGoal = Goal(name="Vacation", targetYear=2027, targetValue=10000, priority="Dreams", portfolio=myPortfolio)
#myAccount=Account(number="123456789", accountType="Taxable", accountStatus=AccountStatus("APPROVED"), cashBalance=11.0)
#myAccount.goals.append(myGoal)

In [ ]:
myPortfolio.calculateDrift()

0.011467735382974804

In [ ]:
diff

[-0.00731313782668816,
 -0.004154597556286707,
 0.0,
 0.007383742507253432,
 0.0040839928757213095]

In [ ]:
# Learn how to implement Time-based Rebalancing 

In [ ]:
splitOrders = []
for i in range(len(diff)):
    if diff[i] > 0:
        diffValue = diff[i] * holdings[i] * market_values[i]
        newOrder = Order(account = myAccount, 
                            goal = myGoal, 
                            transactionType = TransactionType('BUY'), 
                            dollarAmount = diffValue)
        splitOrders.append(SplitOrder(originalOrder = newOrder,
                                      ticker = myPortfolio.allocations[i].ticker, 
                                      dollarAmount = diffValue))
    elif diff[i] < 0:
        diffValue = abs(diff[i]) * holdings[i] * market_values[i]
        newOrder = Order(account = myAccount, 
                            goal = myGoal, 
                            transactionType = TransactionType('SELL'), 
                            dollarAmount = diffValue)
        splitOrders.append(SplitOrder(originalOrder = newOrder,
                                      ticker = myPortfolio.allocations[i].ticker, 
                                      dollarAmount = diffValue))

In [ ]:
splits = myPortfolio.rebalance()

In [ ]:
for split in splits:
  print(split.ticker + ": " + split.originalOrder.transactionType.value + " " + '${0:.2f}'.format(split.dollarAmount))

In [ ]:
# Learn how to implement Threshold-based Rebalancing 

In [ ]:
threshold = 0.003
accounts = [myAccount, myAccount2]
for account in accounts:
  for goal in account.goals:
    diffs = goal.portfolio.calculateDiffsToModel()
    goal.portfolio.checkNeedRebalancing(diff=diffs, thres=threshold)

In [ ]:
print(myAccount.goals[0].portfolio.needRebalancing)
print(myAccount.goals[0].portfolio.calculateDrift())

print(myAccount2.goals[0].portfolio.needRebalancing)
print(myAccount2.goals[0].portfolio.calculateDrift())

True
0.011467735382974804
True
0.01232641795608784


In [ ]:
# Learn how to implement Tax Loss Harvesting 

In [ ]:
history = pd.DataFrame(columns=['Date','Value','Cashflow'])
history = history.set_index('Date')
new_row = pd.Series({'Value':0,'Cashflow':0}, name=pd.Timestamp('2019-12-31'))
history = history.append(new_row)
new_row = pd.Series({'Value':100000,'Cashflow':100000}, name=pd.Timestamp('2019-12-31'))
history = history.append(new_row)
new_row = pd.Series({'Value':77985,'Cashflow':0}, name=pd.Timestamp('2020-03-23'))
history = history.append(new_row)
new_row = pd.Series({'Value':87985,'Cashflow':10000}, name=pd.Timestamp('2020-03-23'))
history = history.append(new_row)
print(history)

             Value Cashflow
Date                       
2019-12-31       0        0
2019-12-31  100000   100000
2020-03-23   77985        0
2020-03-23   87985    10000


In [ ]:
print(calculatePNL_TWRR(history=history))

-0.22015000000000007
